# RHGN - Alibaba dataset process

In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import dgl
import torch
from torch import nn

from utils import *
from utils import neighbormap, split_char, filter_sample, combination

import fasttext

Using backend: pytorch[22:48:06] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/purificato/.local/lib/python3.8/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.1.so: cannot open shared object file: No such file or directory



## Data path

In [2]:
# Get data from the original RHGN path
raw_data_path = '/home/purificato/papers_code/CIKM21_RHGN/taobao_data'
save_path = './input_ali_data/new_age_split'

In [3]:
def show_df_info(df):
    print(df.info())
    print('####### Repeat ####### \n', df.duplicated().any())
    print('####### Count ####### \n', df.nunique())
    print('####### Example ####### \n',df.head())

def label_statics(label_df, label_list):
    print("####### nCount #######")
    for label in label_list:
        print(label_df[label].value_counts())
    print("####### nPercent #######")
    for label in label_list:
        print(label_df[label].value_counts()/label_df.shape[0])

## USER_PROFILE analysis

df_user == label (CatGCN)

In [4]:
df_user = pd.read_csv(os.path.join(raw_data_path, "user_profile.csv"), usecols=[0,3,4,5,7,8])
df_user.dropna(inplace=True)
df_user.rename(columns={'userid':'uid', 'final_gender_code':'gender','age_level':'age', 'pvalue_level':'buy', 'occupation':'student', 'new_user_class_level ':'city'}, inplace=True)

show_df_info(df_user)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395932 entries, 1 to 1061767
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   uid      395932 non-null  int64  
 1   gender   395932 non-null  int64  
 2   age      395932 non-null  int64  
 3   buy      395932 non-null  float64
 4   student  395932 non-null  int64  
 5   city     395932 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 21.1 MB
None
####### Repeat ####### 
 False
####### Count ####### 
 uid        395932
gender          2
age             7
buy             3
student         2
city            4
dtype: int64
####### Example ####### 
     uid  gender  age  buy  student  city
1   523       2    2  1.0        1   2.0
5  3644       2    6  2.0        0   2.0
6  5777       2    5  2.0        0   2.0
8  6355       2    1  1.0        0   4.0
9  6823       2    5  2.0        0   1.0


In [5]:
label_statics(df_user, df_user.columns[1:])

####### nCount #######
2    269064
1    126868
Name: gender, dtype: int64
3    120071
4     94831
2     83409
5     73721
1     16734
6      7026
0       140
Name: age, dtype: int64
2.0    242292
1.0    121345
3.0     32295
Name: buy, dtype: int64
0    367856
1     28076
Name: student, dtype: int64
2.0    178624
3.0     96252
4.0     76959
1.0     44097
Name: city, dtype: int64
####### nPercent #######
2    0.679571
1    0.320429
Name: gender, dtype: float64
3    0.303262
4    0.239513
2    0.210665
5    0.186196
1    0.042265
6    0.017745
0    0.000354
Name: age, dtype: float64
2.0    0.611954
1.0    0.306479
3.0    0.081567
Name: buy, dtype: float64
0    0.929089
1    0.070911
Name: student, dtype: float64
2.0    0.451148
3.0    0.243102
4.0    0.194374
1.0    0.111375
Name: city, dtype: float64


### Label filter

In [6]:
df_user = pd.read_csv(os.path.join(raw_data_path, 'user_profile.csv'), usecols=[0,3,4,5,7,8])
df_user.dropna(inplace=True)
df_user.rename(columns={'userid':'uid', 'final_gender_code':'gender','age_level':'age', 'pvalue_level':'buy', 'occupation':'student', 'new_user_class_level ':'city'}, inplace=True)

show_df_info(df_user)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395932 entries, 1 to 1061767
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   uid      395932 non-null  int64  
 1   gender   395932 non-null  int64  
 2   age      395932 non-null  int64  
 3   buy      395932 non-null  float64
 4   student  395932 non-null  int64  
 5   city     395932 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 21.1 MB
None
####### Repeat ####### 
 False
####### Count ####### 
 uid        395932
gender          2
age             7
buy             3
student         2
city            4
dtype: int64
####### Example ####### 
     uid  gender  age  buy  student  city
1   523       2    2  1.0        1   2.0
5  3644       2    6  2.0        0   2.0
6  5777       2    5  2.0        0   2.0
8  6355       2    1  1.0        0   4.0
9  6823       2    5  2.0        0   1.0


In [7]:
label_statics(df_user, df_user.columns[1:])

####### nCount #######
2    269064
1    126868
Name: gender, dtype: int64
3    120071
4     94831
2     83409
5     73721
1     16734
6      7026
0       140
Name: age, dtype: int64
2.0    242292
1.0    121345
3.0     32295
Name: buy, dtype: int64
0    367856
1     28076
Name: student, dtype: int64
2.0    178624
3.0     96252
4.0     76959
1.0     44097
Name: city, dtype: int64
####### nPercent #######
2    0.679571
1    0.320429
Name: gender, dtype: float64
3    0.303262
4    0.239513
2    0.210665
5    0.186196
1    0.042265
6    0.017745
0    0.000354
Name: age, dtype: float64
2.0    0.611954
1.0    0.306479
3.0    0.081567
Name: buy, dtype: float64
0    0.929089
1    0.070911
Name: student, dtype: float64
2.0    0.451148
3.0    0.243102
4.0    0.194374
1.0    0.111375
Name: city, dtype: float64


### bin_age

In [8]:
df_user['bin_age'] = df_user['age']
df_user['bin_age'] = df_user['bin_age'].replace(1,0)
df_user['bin_age'] = df_user['bin_age'].replace(2,0)
df_user['bin_age'] = df_user['bin_age'].replace(3,0)
df_user['bin_age'] = df_user['bin_age'].replace(4,1)
df_user['bin_age'] = df_user['bin_age'].replace(5,1)
df_user['bin_age'] = df_user['bin_age'].replace(6,1)

label_statics(df_user, df_user.columns[1:])

####### nCount #######
2    269064
1    126868
Name: gender, dtype: int64
3    120071
4     94831
2     83409
5     73721
1     16734
6      7026
0       140
Name: age, dtype: int64
2.0    242292
1.0    121345
3.0     32295
Name: buy, dtype: int64
0    367856
1     28076
Name: student, dtype: int64
2.0    178624
3.0     96252
4.0     76959
1.0     44097
Name: city, dtype: int64
0    220354
1    175578
Name: bin_age, dtype: int64
####### nPercent #######
2    0.679571
1    0.320429
Name: gender, dtype: float64
3    0.303262
4    0.239513
2    0.210665
5    0.186196
1    0.042265
6    0.017745
0    0.000354
Name: age, dtype: float64
2.0    0.611954
1.0    0.306479
3.0    0.081567
Name: buy, dtype: float64
0    0.929089
1    0.070911
Name: student, dtype: float64
2.0    0.451148
3.0    0.243102
4.0    0.194374
1.0    0.111375
Name: city, dtype: float64
0    0.556545
1    0.443455
Name: bin_age, dtype: float64


In [9]:
df_user['bin_buy'] = df_user['buy']
df_user['bin_buy'] = df_user['bin_buy'].replace(3.0, 2.0)
df_user['bin_buy'] = df_user['bin_buy'].astype('int64')

label_statics(df_user, df_user.columns[1:])

####### nCount #######
2    269064
1    126868
Name: gender, dtype: int64
3    120071
4     94831
2     83409
5     73721
1     16734
6      7026
0       140
Name: age, dtype: int64
2.0    242292
1.0    121345
3.0     32295
Name: buy, dtype: int64
0    367856
1     28076
Name: student, dtype: int64
2.0    178624
3.0     96252
4.0     76959
1.0     44097
Name: city, dtype: int64
0    220354
1    175578
Name: bin_age, dtype: int64
2    274587
1    121345
Name: bin_buy, dtype: int64
####### nPercent #######
2    0.679571
1    0.320429
Name: gender, dtype: float64
3    0.303262
4    0.239513
2    0.210665
5    0.186196
1    0.042265
6    0.017745
0    0.000354
Name: age, dtype: float64
2.0    0.611954
1.0    0.306479
3.0    0.081567
Name: buy, dtype: float64
0    0.929089
1    0.070911
Name: student, dtype: float64
2.0    0.451148
3.0    0.243102
4.0    0.194374
1.0    0.111375
Name: city, dtype: float64
0    0.556545
1    0.443455
Name: bin_age, dtype: float64
2    0.693521
1    0.306479


## ITEM analysis

df_item == pid_cid (CatGCN)

In [10]:
df_item = pd.read_csv(os.path.join(raw_data_path, "ad_feature.csv"))
df_item.dropna(axis=0, subset=['cate_id', 'campaign_id', 'brand'], inplace=True)
df_item.rename(columns={"adgroup_id":"pid", "cate_id":"cid"}, inplace=True)

show_df_info(df_item)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600481 entries, 0 to 846810
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   pid          600481 non-null  int64  
 1   cid          600481 non-null  int64  
 2   campaign_id  600481 non-null  int64  
 3   customer     600481 non-null  int64  
 4   brand        600481 non-null  float64
 5   price        600481 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 32.1 MB
None
####### Repeat ####### 
 False
####### Count ####### 
 pid            600481
cid              6459
campaign_id    320127
customer       195847
brand           99814
price           13439
dtype: int64
####### Example ####### 
       pid   cid  campaign_id  customer     brand   price
0   63133  6406        83237         1   95471.0  170.00
1  313401  6406        83237         1   87331.0  199.00
2  248909   392        83237         1   32233.0   38.00
3  208458   392        83237        

## CLICK analysis

df_click == uid_pid (CatGCN)

In [11]:
df_click = pd.read_csv(os.path.join(raw_data_path, "raw_sample.csv"), usecols=["user", "adgroup_id", "clk"])
df_click.rename(columns={"user":"uid", "adgroup_id":"pid"}, inplace=True)

df_click = df_click[df_click["clk"]>0]

df_click.drop("clk", axis=1, inplace=True)

df_click = df_click[df_click["uid"].isin(df_user["uid"])]
df_click = df_click[df_click["pid"].isin(df_item["pid"])]

df_click.drop_duplicates(inplace=True)

show_df_info(df_click)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335619 entries, 92 to 26557931
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   uid     335619 non-null  int64
 1   pid     335619 non-null  int64
dtypes: int64(2)
memory usage: 7.7 MB
None
####### Repeat ####### 
 False
####### Count ####### 
 uid    148608
pid    103292
dtype: int64
####### Example ####### 
          uid  pid
92    642854  102
257   843732  102
283  1076956  102
300   358193  102
350  1002263  102


## Filter

In [12]:
def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=True)
    count = playcount_groupbyid.size()
    return count

def filter_triplets(tp, user, item, min_uc=0, min_sc=0):
    # Only keep the triplets for users who clicked on at least min_uc items
    if min_uc > 0:
        usercount = get_count(tp, user)
        tp = tp[tp[user].isin(usercount.index[usercount >= min_uc])]
    
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
        itemcount = get_count(tp, item)
        tp = tp[tp[item].isin(itemcount.index[itemcount >= min_sc])]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, user), get_count(tp, item) 
    return tp, usercount, itemcount

In [13]:
# Before filtering
users = set(df_click.uid.tolist())
items = set(df_click.pid.tolist())

print(len(users), len(items))

148608 103292


### Filter "df_click" (item interactions >= 2)

In [14]:
df_click, uid_activity, pid_popularity = filter_triplets(df_click, 'uid', 'pid', min_uc=0, min_sc=2) # min_sc>=2

sparsity = 1. * df_click.shape[0] / (uid_activity.shape[0] * pid_popularity.shape[0])

print("After filtering, there are %d interacton events from %d users and %d items (sparsity: %.4f%%)" % 
      (df_click.shape[0], uid_activity.shape[0], pid_popularity.shape[0], sparsity * 100))

After filtering, there are 278019 interacton events from 135361 users and 45692 items (sparsity: 0.0045%)


In [15]:
show_df_info(df_click)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278019 entries, 92 to 26557917
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   uid     278019 non-null  int64
 1   pid     278019 non-null  int64
dtypes: int64(2)
memory usage: 6.4 MB
None
####### Repeat ####### 
 False
####### Count ####### 
 uid    135361
pid     45692
dtype: int64
####### Example ####### 
          uid  pid
92    642854  102
257   843732  102
283  1076956  102
300   358193  102
350  1002263  102


In [16]:
# After filtering
users = set(df_click.uid.tolist())
items = set(df_click.pid.tolist())

print(len(users), len(items))

135361 45692


In [17]:
df_user = df_user[df_user['uid'].isin(users)]
df_item = df_item[df_item['pid'].isin(items)]
df_user.reset_index(drop=True, inplace=True)
df_item.reset_index(drop=True, inplace=True)

## Save

In [18]:
df_user.to_csv(os.path.join(save_path, "user.csv"), index=False)
df_item.to_csv(os.path.join(save_path, "item.csv"), index=False)
df_click.to_csv(os.path.join(save_path, "click.csv"), index=False)

## Process

In [19]:
df_user = df_user.astype({'uid': 'str'}, copy=False)
df_item = df_item.astype({'pid': 'str', 'cid': 'str', 'campaign_id': 'str', 'brand': 'str'}, copy=False)
df_click = df_click.astype({'uid': 'str', 'pid': 'str'}, copy=False)

In [20]:
# Build a dictionary and remove duplicate items
user_dic = {k: v for v, k in enumerate(df_user.uid)}
cate_dic = {k: v for v, k in enumerate(df_item.cid.drop_duplicates())}
campaign_dic = {k: v for v, k in enumerate(df_item.campaign_id.drop_duplicates())}
brand_dic = {k: v for v, k in enumerate(df_item.brand.drop_duplicates())}

In [21]:
item_dic = {}
c1, c2, c3=[],[],[]
for i in range(len(df_item)):
    k=df_item.at[i,'pid']
    v=i
    item_dic[k]=v
    c1.append(cate_dic[df_item.at[i,'cid']])
    c2.append(campaign_dic[df_item.at[i,'campaign_id']])
    c3.append(brand_dic[df_item.at[i,'brand']])

print(min(c1), min(c2), min(c3))
print(len(cate_dic), len(campaign_dic), len(brand_dic))

0 0 0
2590 38215 20486


In [22]:
df_click=df_click[df_click['pid'].isin(item_dic)]
df_click=df_click[df_click['uid'].isin(user_dic)]
df_click.reset_index(drop=True, inplace=True)

## Generate graph

In [23]:
click_user = [user_dic[user] for user in df_click.uid]
click_item = [item_dic[item] for item in df_click.pid]

In [24]:
data_dict = {
    ("user", "click", "item"): (torch.tensor(click_user), torch.tensor(click_item)),
    ("item", "click_by", "user"): (torch.tensor(click_item), torch.tensor(click_user))
}

In [25]:
G = dgl.heterograph(data_dict)

### Process with fasttext model 

In [26]:
model = fasttext.load_model('/home/purificato/papers_code/CIKM21_RHGN/jd_data/fasttext/fastText/cc.zh.200.bin')

temp1 = {k: model.get_sentence_vector(v) for v,k in cate_dic.items()}
cid1_feature = torch.tensor([temp1[k] for _, k in cate_dic.items()])

temp2 = {k: model.get_sentence_vector(v) for v, k in campaign_dic.items()}
cid2_feature = torch.tensor([temp2[k] for _, k in campaign_dic.items()])

temp3 = {k: model.get_sentence_vector(v) for v, k in brand_dic.items()}
cid3_feature = torch.tensor([temp3[k] for _, k in brand_dic.items()])

/tmp/ipykernel_49325/3635140941.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  cid1_feature = torch.tensor([temp1[k] for _, k in cate_dic.items()])


In [27]:
def col_map(df, col, num2id):
    df[[col]] = df[[col]].applymap(lambda x: num2id[x])
    return df

def label_map(label_df, label_list):
    for label in label_list:
        label2id = {num: i for i, num in enumerate(pd.unique(label_df[label]))}
        label_df = col_map(label_df, label, label2id)
    return label_df

In [28]:
uid2id = {num: i for i, num in enumerate(df_user['uid'])}

df_user = col_map(df_user, 'uid', uid2id)
user_label = label_map(df_user, df_user.columns[1:])

show_df_info(user_label)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135361 entries, 0 to 135360
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   uid      135361 non-null  int64
 1   gender   135361 non-null  int64
 2   age      135361 non-null  int64
 3   buy      135361 non-null  int64
 4   student  135361 non-null  int64
 5   city     135361 non-null  int64
 6   bin_age  135361 non-null  int64
 7   bin_buy  135361 non-null  int64
dtypes: int64(8)
memory usage: 8.3 MB
None
####### Repeat ####### 
 False
####### Count ####### 
 uid        135361
gender          2
age             7
buy             3
student         2
city            4
bin_age         2
bin_buy         2
dtype: int64
####### Example ####### 
    uid  gender  age  buy  student  city  bin_age  bin_buy
0    0       0    0    0        0     0        0        0
1    1       0    1    1        1     0        1        1
2    2       0    2    1        1     0        1        1
3    3 

In [29]:
label_statics(user_label, user_label.columns[1:])

####### nCount #######
0    99593
1    35768
Name: gender, dtype: int64
4    41335
3    33023
0    28088
2    24845
5     5736
1     2289
6       45
Name: age, dtype: int64
1    83120
0    41529
2    10712
Name: buy, dtype: int64
1    126080
0      9281
Name: student, dtype: int64
0    61134
3    33485
2    26504
1    14238
Name: city, dtype: int64
0    75204
1    60157
Name: bin_age, dtype: int64
1    93832
0    41529
Name: bin_buy, dtype: int64
####### nPercent #######
0    0.735758
1    0.264242
Name: gender, dtype: float64
4    0.305369
3    0.243962
0    0.207504
2    0.183546
5    0.042376
1    0.016910
6    0.000332
Name: age, dtype: float64
1    0.614062
0    0.306802
2    0.079137
Name: buy, dtype: float64
1    0.931435
0    0.068565
Name: student, dtype: float64
0    0.451637
3    0.247376
2    0.195802
1    0.105185
Name: city, dtype: float64
0    0.555581
1    0.444419
Name: bin_age, dtype: float64
1    0.693198
0    0.306802
Name: bin_buy, dtype: float64


In [ ]:
# Pass the label into "label"
label_gender = user_label.gender
label_age = user_label.age
label_buy = user_label.buy
label_student = user_label.student
label_city = user_label.city
label_bin_buy = user_label.bin_buy

In [ ]:
G.nodes['user'].data['gender'] = torch.tensor(label_gender[:G.number_of_nodes('user')])
G.nodes['user'].data['age'] = torch.tensor(label_age[:G.number_of_nodes('user')])
G.nodes['user'].data['buy'] = torch.tensor(label_buy[:G.number_of_nodes('user')])
G.nodes['user'].data['student'] = torch.tensor(label_student[:G.number_of_nodes('user')])
G.nodes['user'].data['city'] = torch.tensor(label_city[:G.number_of_nodes('user')])
G.nodes['user'].data['bin_buy'] = torch.tensor(label_bin_buy[:G.number_of_nodes('user')])

G.nodes['item'].data['cid1'] = torch.tensor(c1[:G.number_of_nodes('item')])
G.nodes['item'].data['cid2'] = torch.tensor(c2[:G.number_of_nodes('item')])
G.nodes['item'].data['cid3'] = torch.tensor(c3[:G.number_of_nodes('item')])

print(G)
print(cid1_feature.shape,)
print(cid2_feature.shape,)
print(cid3_feature.shape,)

In [ ]:
torch.save(G, os.path.join(save_path, "G_new.pkl"))
torch.save(cid1_feature, os.path.join(save_path, "cid1_feature.npy"))
torch.save(cid2_feature, os.path.join(save_path, "cid2_feature.npy"))
torch.save(cid3_feature, os.path.join(save_path, "cid3_feature.npy"))